In [ ]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL
import math
import matplotlib.gridspec as gridspec
from sklearn.tree import DecisionTreeClassifier

import data_transforations as data_tr

from sklearn import tree

# parameters

In [ ]:
current_date='2025-09-01'
model_data = 'at_inception' #'near_future'/'at_inception'

# loading data

In [3]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/Pipeline_test/"

calls_data = pd.DataFrame(pd.read_csv(path+"calls.csv"))
sales_data = pd.DataFrame(pd.read_csv(path+"sales.csv"))
policy_data = pd.DataFrame(pd.read_csv(path+"policy.csv"))
payment_data = pd.DataFrame(pd.read_csv(path+"payment.csv"))
lapse_data = pd.DataFrame(pd.read_csv(path+"lapse.csv"))
#payment_data = pd.concat([pd.DataFrame(pd.read_csv(path+"Payment_data_1.csv")),pd.DataFrame(pd.read_csv(path+"Payment_data_2.csv"))])
#lapse_data = pd.concat([pd.DataFrame(pd.read_csv(path+"lapse_1.csv")),pd.DataFrame(pd.read_csv(path+"lapse_2.csv"))])

# Processing data

 Filtering data to relevant policies

Relevant policies are policies that have been incepted. The rpt_datamodel_lapse has incepted policies only, hence filtering on policies within that dataset

loading economic variables

In [ ]:
pol_names = list(lapse_data['policy_name'])
sales_data = sales_data[sales_data['policy_name'].isin(pol_names)]
pol_ids= list(sales_data['policy_id'])
calls_data = calls_data[calls_data['policy_name'].isin(pol_names)]
payment_data = payment_data[payment_data['policy_id'].isin(pol_ids)]
policy_data = policy_data[policy_data['policy_name'].isin(pol_names)]

#Make date column of data type datetime
sales_data=do.dtype_datetime(sales_data)
calls_data=do.dtype_datetime(calls_data)
payment_data=do.dtype_datetime(payment_data)
policy_data=do.dtype_datetime(policy_data)
lapse_data=do.dtype_datetime(lapse_data)

eco_path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"
unemployment = pd.DataFrame(pd.read_csv(eco_path+"unemployment rates.csv"))
unemployment= unemployment[['year','ave']]
inflation = pd.DataFrame(pd.read_csv(eco_path+"inflation.csv"))


### Aggregating call, payment, sales and lapse data then merging. Cleaning the data, adding economic variables, creating targets, grouping continous variable (for at_inception) and saving data as csv to specified path under parameters

In [ ]:
if model_data =='at_inception':
    pre_inception_calls_data = data_tr.pre_sale_calls(calls_data,policy_data)
    lapse_data = data_tr.lapses(lapse_data)
    sales_data = sales_data.drop_duplicates(subset=['policy_id'], keep='last')
    inception_data = data_tr.at_inception_data_merge(pre_inception_calls_data, sales_data,policy_data,lapse_data)
    inception_data = data_tr.at_inception_data_clean_filter(inception_data)

    inception_data['sale_date_year'] = pd.to_datetime(inception_data['sale_date']).dt.year
    inception_data = data_tr.adding_eco_ind(inflation, unemployment,inception_data,'sale_date_year')

    inception_data = data_tr.inception_targets(inception_data)

    inception_data.drop(columns=['orginal individual_income', 'policy_id', 'sale_date_year', 'policy_type',
           'campaign_name', 'cancellation_effective_date', 'lapse_type','cover_start_date', 'end_date', 'pol_duration'],inplace=True)
    cont_var_list=['age','income',  'last_benefit_amount', 'cover_amount_full', 'total_funeral_premium',  'original_premium']
    target='lapse_flag'
    df=inception_data.copy()
    for variable in cont_var_list:
        X = df[variable].values.reshape(-1, 1)
        Y = df[target]
        clf=tree.DecisionTreeClassifier(min_samples_leaf=math.floor(0.15*inception_data.shape[0]))
        clf=clf.fit(X,Y)
        df[variable+'_band'] = data_tr.group_by_decision_tree(clf, df[variable], df)
    inception_data=df
    inception_data.to_csv(path+'at_incpt_data.csv')

elif model_data=='near_future':
    post_sale_calls_data = data_tr.post_sale_calls(calls_data,policy_data)
    payment_hist_data = data_tr.payments_history(payment_data,current_date=current_date)
    lapse_data = data_tr.lapses(lapse_data)
    sales_data = sales_data.drop_duplicates(subset=['policy_id'], keep='last')
    near_ftr_lapse_data = data_tr.post_sale_data_merge(post_sale_calls_data, sales_data,policy_data,lapse_data,payment_hist_data)
    near_ftr_lapse_data = data_tr.near_ftr_lapse_data_clean_filter(near_ftr_lapse_data)

    near_ftr_lapse_data['cover_start_date_year' ] = pd.to_datetime(near_ftr_lapse_data['cover_start_date' ]).dt.year
    near_ftr_lapse_data = data_tr.adding_eco_ind(inflation, unemployment,near_ftr_lapse_data,'cover_start_date_year')

    near_ftr_lapse_data = data_tr.near_ftr_targets(near_ftr_lapse_data)
    near_ftr_lapse_data.to_csv(path+'near_ftr_model_data.csv')


c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories)
c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories_postsale)
c:\Users\E1005279\OneD